In [1]:
import pandas as pd
import numpy as np
from early_fusion import *
from tqdm import tqdm

c:\Users\Justin\Documents\JKU\2024WS\multimedia-search-and-retrieval\.venv\Lib\site-packages\tqdm_joblib\__init__.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
retrieval = MultiModalEarlyFusionRetrieval()

feature_files = {
    'mfcc_bow': 'dataset/id_mfcc_bow_mmsr.tsv',
    #'mfcc_stats': 'dataset/id_mfcc_stats_mmsr.tsv',

    #'blf_correlation' : 'dataset/id_blf_correlation_mmsr.tsv',
    #'blf_spectral': 'dataset/id_blf_spectral_mmsr.tsv',
    #'blf_spectralcontrast': 'dataset/id_blf_spectralcontrast_mmsr.tsv',
    #'blf_deltaspectral': 'dataset/id_blf_deltaspectral_mmsr.tsv',
    #'blf_logfluc': 'dataset/id_blf_logfluc_mmsr.tsv',
    #'blf_vardeltaspectral': 'dataset/id_blf_vardeltaspectral_mmsr.tsv',

    'musicnn': 'dataset/id_musicnn_mmsr.tsv',

    'lyrics_bert': 'dataset/id_lyrics_bert_mmsr.tsv',
    #'lyrics_tf-idf': 'dataset/id_lyrics_tf-idf_mmsr.tsv',
    #'lyrics_word2vec': 'dataset/id_lyrics_word2vec_mmsr.tsv',

    'resnet': 'dataset/id_resnet_mmsr.tsv',
    #'vgg': 'dataset/id_vgg19_mmsr.tsv'
    #'incp': 'dataset/id_incp_mmsr.tsv'
}

retrieval.load_and_normalize_features(feature_files)

feature_weights = [
    ('mfcc_bow', 0.4),
    #('mfcc_stats', 0.4),

    #('blf_correlation', 0.4),
    #('blf_spectral', 0.4),
    #('blf_spectralcontrast', 0.4),
    #('blf_deltaspectral', 0.4),
    #('blf_logfluc', 0.4),
    #('blf_vardeltaspectral', 0.4),

    ('musicnn', 0.4),

    ('lyrics_bert', 0.4),
    #('lyrics_tf-idf', 0.4),
    #('lyrics_word2vec', 0.4),

    ('resnet', 0.4),
    #('vgg', 0.4),
    #('incp', 0.4)
]

retrieval.fuse_features(feature_weights)

infos = pd.read_csv("dataset/id_information_mmsr.tsv", sep="\t")

song_ids = infos['id'].tolist()
results = retrieval.batch_similarity_search(song_ids, top_k=10)

Using 4 cores for batch similarity search.


Performing Similarity Search: 10296it [09:04, 18.93it/s]                         


In [6]:
results_df = pd.DataFrame(results)
results_df.to_csv('./predictions/rets_early_fusion_results_ui_10.csv', sep=",", index=False, header=True)

In [8]:
inter_matr = np.zeros((len(infos), len(infos)))


song_ids = infos['id'].tolist()
song_id_to_index = {song_id: idx for idx, song_id in enumerate(song_ids)}

for idx, row in tqdm(results_df.iterrows(), desc="generate matrix", total=len(results_df)):
    source = row["source_id"]
    target = row["target_id"]
    inter_matr[song_id_to_index[source], song_id_to_index[target]] = row["similarity"]

np.savetxt(f"./predictions/rets_early_fusion_matrix_10.csv", inter_matr, delimiter=",")

generate matrix: 100%|██████████| 51480/51480 [00:02<00:00, 19247.09it/s]
